In [6]:
import numpy as np
import pandas as pd 

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

import subprocess
import json

In [7]:
df = pd.read_csv("D:\per\Codes\cloud_deployment\mlops\data\creditcard.csv\creditcard.csv")

In [21]:
# define query() function that will get the model predictions

def query(input_json):
    proc = subprocess.run(["curl", "-X", "POST", "-H", 
                           "Content-Type:application/json; format=pandas-split", 
                           "--data", input_json, 
                           "http://34.118.241.159:5000/invocations"], 
                           stdout=subprocess.PIPE, encoding='utf-8')
    output = proc.stdout 
    preds = json.loads(output)
    return preds

* Now for the IP taking the form `http://VM'S_EXTERNAL_IP:5000/invocations`

Querying the Model: 


In [22]:
input_json = df.iloc[:80].drop(["Time", "Class"], axis=1).to_json(orient="split")
pd.DataFrame(query(input_json)).T

,0,1,2,3,4,5,6,7,8,9,...,70,71,72,73,74,75,76,77,78,79
0,1,0,1,1,1,0,0,1,1,0,...,1,0,0,0,0,0,1,1,0,0


* output of querying model in the gcloud bucket with the first 80 rows of the dataframe
* as expected not the greatest of predictins since we haven't scaled anything yet :)

### Scaling our values

In [23]:
normal = df[df.Class == 0].sample(frac=0.5, random_state=2020).reset_index(drop=True)
anomaly = df[df.Class == 1]

normal_train, normal_test = train_test_split(normal, test_size=0.2, random_state=2020)
anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.2, random_state=2020)

In [24]:
scaler = StandardScaler()
scaler.fit(pd.concat((normal, anomaly)).drop(["Time", "Class"], axis=1))


StandardScaler(copy=True, with_mean=True, with_std=True)

In [25]:
test = pd.concat((normal.iloc[:1900], anomaly.iloc[:100]))
true = test.Class  

test = scaler.transform(test.drop(["Time", "Class"], axis=1))

preds = []

batch_size = 80
for f in range(25):
    print(f"Batch {f}", end=" - ")

    sample = pd.DataFrame(test[f*batch_size:(f+1)*batch_size]).to_json(orient="split")

    output = query(sample)
    resp = pd.DataFrame([output])
    preds = np.concatenate((preds, resp.values[0]))

    print("Completed")

eval_acc = accuracy_score(true, preds)
eval_auc = roc_auc_score(true, preds)

print("Eval Acc", eval_acc)
print("Eval AUC", eval_auc)

Batch 0 - Completed
Batch 1 - Completed
Batch 2 - Completed
Batch 3 - Completed
Batch 4 - Completed
Batch 5 - Completed
Batch 6 - Completed
Batch 7 - Completed
Batch 8 - Completed
Batch 9 - Completed
Batch 10 - Completed
Batch 11 - Completed
Batch 12 - Completed
Batch 13 - Completed
Batch 14 - Completed
Batch 15 - Completed
Batch 16 - Completed
Batch 17 - Completed
Batch 18 - Completed
Batch 19 - Completed
Batch 20 - Completed
Batch 21 - Completed
Batch 22 - Completed
Batch 23 - Completed
Batch 24 - Completed
Eval Acc 0.9915
Eval AUC 0.915


* results of the batch query script